In [ ]:
import os
import json

from PIL import Image

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torchvision import transforms

from sklearn.model_selection import train_test_split

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(0)

from torch.optim import Adam
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from livelossplot import PlotLosses

import ternausnet.models
# model = ternausnet.models.UNet11(pretrained=True)

In [ ]:
root_path = 'datasets/landcover_processed/rotated_crops/'
list_dataset = list(filter(lambda x: x.endswith('.png'), os.listdir(root_path)))
list_dataset = list(map(lambda x: root_path + x, list_dataset))
list_dataset = list_dataset[:1000]
#train_img_list, test_img_list = train_test_split(list_dataset, test_size=0.15)
#test_img_list = list(filter(lambda x: x.endswith('-0.png'), test_img_list))

In [ ]:
def root(name):
    return "-".join(name.split("/")[-1].split(".")[-2].split("-")[:-2 or None])

In [ ]:
list_root = [root(name) for name in list_dataset]
list_root = list(set(list_root))
len(list_root)

In [ ]:
train_root_list, test_root_list = train_test_split(list_root, test_size=0.15)

In [ ]:
len(train_root_list), len(test_root_list)

In [ ]:
train_img_list = []
test_img_list = []
for name in list_dataset:
    if root(name) in test_root_list and name.endswith('-0.png'):
        test_img_list.append(name)
    else:
        train_img_list.append(name)

In [ ]:
len(train_img_list), len(test_img_list)

In [ ]:
print(len(train_img_list), len(test_img_list))
with open('train_img_list.json', 'w') as f:
    json.dump(train_img_list, f)
    
with open('test_img_list.json', 'w') as f:
    json.dump(test_img_list, f)

In [ ]:
from lib.train import WaterDataset, viz, train_transform, test_transform

In [ ]:
d_train = WaterDataset('train_img_list.json', train_transform)
d_val = WaterDataset('test_img_list.json', test_transform)

In [ ]:
len(d_train)

In [ ]:
%pylab inline

In [ ]:
def viz2(s):
    img = (s * torch.tensor([0.229, 0.224, 0.225])[:, None, None] + torch.tensor([0.485, 0.456, 0.406])[:, None,None])
    print(img.min(), img.max(), img.shape)
    img = (255 * (img.transpose(0,1).transpose(1,2).numpy())).astype(np.uint8)
    print(img.shape)
    plt.imshow(img)
    return img

In [ ]:
d_val[10][1].shape

In [ ]:
II = viz2(d_val[10][0])

In [ ]:
II.shape

In [ ]:
import json
with open('test_img_list.json', 'r') as f:
    path = json.load(f)[10]

In [ ]:
import cv2
I = cv2.imread('datasets/landcover_processed/rotated_crops/N-34-66-C-c-4-3-773-60.png', cv2.IMREAD_UNCHANGED)

In [ ]:
from PIL import Image
I = Image.open('datasets/landcover_processed/rotated_crops/N-34-66-C-c-4-3-773-60.png',).resize((224,224))

In [ ]:
#plt.hist((II.astype(np.int32)-I[:,:,[2,1,0]].astype(np.int32)).reshape(-1))

In [ ]:
#plt.imshow(II-I[:,:,[2,1,0]])

In [ ]:
I.mode

In [ ]:
#I = cv2.resize(I, (224,224), interpolation=cv2.INTER_LINEAR)

In [ ]:
#I.shape

In [ ]:
#I[:,:,[-1]].shape

In [ ]:
#plt.imshow(I[:, :, [2,1,0]])

In [ ]:
d_val[3]

In [ ]:
acc = AccuracyMetric(0.5)

pred = torch.tensor([0.6,0.4,0.1])
gt = torch.tensor([1,0,1])

acc.append(pred, gt)

pred = torch.tensor([0.6,0.4,0.1])
gt = torch.tensor([1,0,0])


acc.append(pred, gt)

acc.result()

In [ ]:
with open('train_config.json', 'r') as f:
    cfg = json.load(f)

In [ ]:
# groups = {'acccuracy': ['acc', 'val_acc'], 'bce-loss': ['loss', 'val_loss'], 'lake-acc': ['lakeacc', 'val_lakeacc']}
# plotlosses = PlotLosses(groups=groups)

# groups = {'acccuracy': ['train_acc', 'val_acc']}
# plotlosses = PlotLosses(groups=groups)

In [ ]:
trainer(cfg['from_scratch'], train_id="scratch")

In [ ]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print(torch.cuda.is_available())